In [1]:
import numpy as np
import cv2
import os

from keras.models import Sequential, Model
from keras.applications.vgg16 import VGG16, preprocess_input
from keras.preprocessing.image import ImageDataGenerator,load_img, img_to_array
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Dropout, Input, Flatten, SeparableConv2D, Activation
from keras.layers import GlobalMaxPooling2D
from keras.layers.normalization import BatchNormalization
from keras.layers.merge import Concatenate
from keras.models import Model
from keras.optimizers import Adam, SGD, RMSprop
from keras.callbacks import ModelCheckpoint, Callback, EarlyStopping
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split

Using TensorFlow backend.
/opt/conda/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/opt/conda/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/opt/conda/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/opt/conda/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:519: FutureWarn

In [2]:
def getLabel(l):
    if (l == 'NORMAL'):
        return [1,0]
    elif (l == 'PNEUMONIA'):
        return [0,1]
    else:
        return None

def loadPaths(type):
    imgs = []
    labels = []
    for t in os.listdir(os.path.join('/kaggle/input/chest-xray-pneumonia/chest_xray/chest_xray',type)):
        if t != '.DS_Store':
            for i in os.listdir(os.path.join('/kaggle/input/chest-xray-pneumonia/chest_xray/chest_xray',type,t)):
                if i != '.DS_Store':
                   imgs.append(os.path.join('/kaggle/input/chest-xray-pneumonia/chest_xray/chest_xray',type,t,i))
                   if getLabel(t) != None:
                      labels.append(getLabel(t))


            # imgs.append(cv2.imread('image',-1))
    return imgs,labels

def loadData():
    data = []
    lbls = []
    cats = ['test','train','val']
    for i in cats:
        pth,labels = loadPaths(i)
        array = []
        for j in pth:
            if not '.DS_Store' in j:
               img = cv2.resize(cv2.imread(j,1),(224,224))
               array.append(img)
        data.append(np.array(array))
        lbls.append(np.array(labels))
        print(i + 'done')
    return data,lbls


In [3]:
def build_model():
    input_img = Input(shape=(224,224,3), name='ImageInput')
    x = Conv2D(64, (3,3), activation='relu', padding='same', name='Conv1_1')(input_img)
    x = Conv2D(64, (3,3), activation='relu', padding='same', name='Conv1_2')(x)
    x = MaxPooling2D((2,2), name='pool1')(x)

    x = SeparableConv2D(128, (3,3), activation='relu', padding='same', name='Conv2_1')(x)
    x = SeparableConv2D(128, (3,3), activation='relu', padding='same', name='Conv2_2')(x)
    x = MaxPooling2D((2,2), name='pool2')(x)

    x = SeparableConv2D(256, (3,3), activation='relu', padding='same', name='Conv3_1')(x)
    x = BatchNormalization(name='bn1')(x)
    x = SeparableConv2D(256, (3,3), activation='relu', padding='same', name='Conv3_2')(x)
    x = BatchNormalization(name='bn2')(x)
    x = SeparableConv2D(256, (3,3), activation='relu', padding='same', name='Conv3_3')(x)
    x = MaxPooling2D((2,2), name='pool3')(x)

    x = SeparableConv2D(512, (3,3), activation='relu', padding='same', name='Conv4_1')(x)
    x = BatchNormalization(name='bn3')(x)
    x = SeparableConv2D(512, (3,3), activation='relu', padding='same', name='Conv4_2')(x)
    x = BatchNormalization(name='bn4')(x)
    x = SeparableConv2D(512, (3,3), activation='relu', padding='same', name='Conv4_3')(x)
    x = MaxPooling2D((2,2), name='pool4')(x)

    x = Flatten(name='flatten')(x)
    x = Dense(1024, activation='relu', name='fc1')(x)
    x = Dropout(0.7, name='dropout1')(x)
    x = Dense(512, activation='relu', name='fc2')(x)
    x = Dropout(0.5, name='dropout2')(x)
    x = Dense(2, activation='softmax', name='fc3')(x)

    model = Model(inputs=input_img, outputs=x)
    return model

In [4]:
[testx,trainx,valx],[testy,trainy,valy] = loadData()

testdone
traindone
valdone


In [5]:
model =  build_model()

opt = Adam(lr=0.0001, decay=1e-5)
es = EarlyStopping(patience=5)
chkpt = ModelCheckpoint(filepath='output', save_best_only=True, save_weights_only=True)
model.compile(loss='binary_crossentropy', metrics=['accuracy'],optimizer=opt)

batch_size = 16
nb_epochs = 5

nb_train_steps = trainx.shape[0]//batch_size
nb_val_steps = valx.shape[0]//batch_size


In [6]:
model.fit(trainx,trainy,epochs = nb_epochs,validation_data=(valx, valy),callbacks=[es, chkpt])
model.save('model.h5')

Train on 5216 samples, validate on 16 samples
Epoch 1/5
5216/5216 [==============================] - 68s 13ms/step - loss: 0.2527 - acc: 0.8867 - val_loss: 0.8863 - val_acc: 0.6250
Epoch 2/5
5216/5216 [==============================] - 58s 11ms/step - loss: 0.0970 - acc: 0.9657 - val_loss: 0.4647 - val_acc: 0.6875
Epoch 3/5
5216/5216 [==============================] - 58s 11ms/step - loss: 0.0767 - acc: 0.9711 - val_loss: 0.2298 - val_acc: 0.8750
Epoch 4/5
5216/5216 [==============================] - 58s 11ms/step - loss: 0.0549 - acc: 0.9804 - val_loss: 0.0716 - val_acc: 1.0000
Epoch 5/5
5216/5216 [==============================] - 58s 11ms/step - loss: 0.0471 - acc: 0.9822 - val_loss: 1.8232 - val_acc: 0.5625


In [7]:
score, acc = model.evaluate(testx, testy)
print('Test score:', score)
print('Test accuracy:', acc)

624/624 [==============================] - 2s 3ms/step
Test score: 2.302905326032977
Test accuracy: 0.6586538461538461
